In [1]:
using Distributed

In [2]:
addprocs(4)

4-element Vector{Int64}:
 2
 3
 4
 5

In [9]:
@everywhere function getvec(x::Int, y::Int)
    vec::Vector{Int} = []
    for i in 1:5
        push!(vec, (myid()+i+x+y)^2)
    end
    return vec
end

In [10]:
futures = Future[]
for i in workers()
    push!(futures, remotecall(getvec, i, 0, 0))
end

In [11]:
results = []
for future in futures
    push!(results, fetch(future))
end

In [12]:
@show results

results = Any[[9, 16, 25, 36, 49], [16, 25, 36, 49, 64], [25, 36, 49, 64, 81], [36, 49, 64, 81, 100]]


4-element Vector{Any}:
 [9, 16, 25, 36, 49]
 [16, 25, 36, 49, 64]
 [25, 36, 49, 64, 81]
 [36, 49, 64, 81, 100]

In [8]:
nworkers()

4

In [15]:
include("AZP_env.jl")
include("AZP_agt.jl")
include("AZP_mcts_valMAX.jl")

check_RL (generic function with 1 method)

In [5]:
env0 = init_Env(["9", "240", "128", "12", "200", "48", "1", "10", "0.3", "0.25", "50", "4", "15.0", "0.4", "1.0", "0.7", "0.5", "4", "0.75", "0.000001", "12.0", "1.0"])

max_turn:  9
num_player:  240
middle_dim:  128
depth:  12
training_step:  200
batch_size:  48
batch_num:  1
num_simulation:  10
α:  0.3
frac:  0.25
ratio:  12.0
ratio_r:  1.0


Env(9, 240, 2, 3, 1, 6, 54, 128, 7, 12, 200, 200, 48, 1, 1.0f-5, 0.9f0, 10, 0.3f0, 0.25f0, 12.0f0, 1.0f0, 50, 4, 15.0f0, 0.4f0, 1.0f0, 0.7f0, 0.5f0, ComplexF32[-2.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.7f0 + 0.0f0im; 0.0f0 - 0.0f0im 1.0f0 + 0.0f0im -0.7f0 + 0.0f0im 0.0f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.7f0 - 0.0f0im 1.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.7f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im], ComplexF32[0.0f0 + 0.0f0im -0.4f0 + 0.0f0im -0.4f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.4f0 - 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.4f0 + 0.0f0im; -0.4f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im -0.4f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.4f0 - 0.0f0im -0.4f0 - 0.0f0im 0.0f0 + 0.0f0im], 0.00837758f0, 4, 0.75f0, 1.0f-6)

In [6]:
agt0 = init_agt()

Agent(Int64[], Float32[], [-1], Vector{Float32}[])

In [8]:
tanh10(x) = Float32(12)*tanh(x/10)
tanh2(x) = Float32(4)*tanh(x/4)

tanh2 (generic function with 1 method)

In [9]:
using BSON: @save, @load
@load "/home/yoshihiro/Documents/Codes/julia/AlphaZeroForPhysics/AZP_valMAX_s_head1.bson" model0

In [13]:
model1 = gpu(model0)

Chain(
  Dense(54 => 256),                     # 14_080 parameters
  BatchNorm(256),                       # 512 parameters, plus 512
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(256),                 # 512 parameters, plus 512
        Dense(256 => 256, relu),        # 65_792 parameters
        Dense(256 => 256, relu),        # 65_792 parameters
      ),
      identity,
    ),
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(256),                 # 512 parameters, plus 512
        Dense(256 => 256, relu),        # 65_792 parameters
        Dense(256 => 256, relu),        # 65_792 parameters
      ),
      identity,
    ),
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(256),                 # 512 parameters, plus 512
        Dense(256 => 256, relu),        # 65_792 parameters
        Dense(256 => 256, relu),        # 65_792 parameters
      ),
      identity,
    ),
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchN

In [16]:
scores = Dict{Vector{Int}, Float32}()

Dict{Vector{Int64}, Float32}()

In [17]:
action, root, surp = run_MCTS_withS(env0, agt0, model1, env0.ratio, env0.ratio_r, scores)

isfinish?: true
has_children?: false
isfinish?: true
has_children?: false
isfinish?: false
has_children?: false
[3]
isfinish?: true
has_children?: false
isfinish?: false
has_children?: false
[3, 6]
isfinish?: true
has_children?: false
isfinish?: false
has_children?: false
[5]
isfinish?: false
has_children?: false
[5, 3]
isfinish?: false
has_children?: false
[3, 6, 5]
isfinish?: false
has_children?: false
[6]


(1, Node(10, 0.0f0, 6.830665f0, Dict{Int64, Node}(5 => Node(2, 0.15226936f0, 3.9492064f0, Dict{Int64, Node}(4 => Node(0, 0.116853885f0, 0.0f0, Dict{Int64, Node}(), 0), 6 => Node(0, 0.30972093f0, 0.0f0, Dict{Int64, Node}(), 0), 2 => Node(0, 0.10977959f0, 0.0f0, Dict{Int64, Node}(), 0), 3 => Node(1, 0.37371558f0, 1.5548325f0, Dict{Int64, Node}(5 => Node(0, 0.41720852f0, 0.0f0, Dict{Int64, Node}(), 0), 4 => Node(0, 0.2261803f0, 0.0f0, Dict{Int64, Node}(), 0), 6 => Node(0, 0.05711125f0, 0.0f0, Dict{Int64, Node}(), 0), 2 => Node(0, 0.23829201f0, 0.0f0, Dict{Int64, Node}(), 0), 1 => Node(0, 0.06120792f0, 0.0f0, Dict{Int64, Node}(), 0)), 0), 1 => Node(0, 0.08993f0, 0.0f0, Dict{Int64, Node}(), 0)), 0), 4 => Node(0, 0.049558394f0, 0.0f0, Dict{Int64, Node}(), 0), 6 => Node(1, 0.13581413f0, 6.830665f0, Dict{Int64, Node}(5 => Node(0, 0.10327234f0, 0.0f0, Dict{Int64, Node}(), 0), 4 => Node(0, 0.039665285f0, 0.0f0, Dict{Int64, Node}(), 0), 2 => Node(0, 0.05450576f0, 0.0f0, Dict{Int64, Node}(), 0), 3